In [1]:
import os 

os.chdir(os.path.abspath(os.path.pardir))

In [ ]:
from app.clients.frame_finder import FrameFinder
from app.metrics.iou import IOUCalculator
from app.metrics.map import MAPCalculator
from app.metrics.recognition_metrics import VideoIdentificationEvaluator

In [ ]:
# Init frame finder

json_files=[]
folder_path = "results/aaivoninskaya avzotova_4 anaalesuslova"

for filename in os.listdir(folder_path):
    full_path = os.path.join(folder_path, filename)

    if os.path.isfile(full_path) and filename.lower().endswith(".json"):
        json_files.append(full_path)
frame_finder = FrameFinder(json_files=json_files)

In [ ]:
iou_calculator = IOUCalculator("annotations.xml", frame_finder)

stats = iou_calculator.get_summary_stats()

print("Person detection metrics:")
print(f"IoU mean: {stats['person_iou']['mean']:.3f} ± {stats['person_iou']['std']:.3f}")
print(f"Count difference: {stats['person_count_diff']['mean']:.1f}")

print("\nFace detection metrics:")
print(f"IoU mean: {stats['face_iou']['mean']:.3f} ± {stats['face_iou']['std']:.3f}")
print(f"Count difference: {stats['face_count_diff']['mean']:.1f}")

Person detection metrics:
IoU mean: 0.764 ± 0.078
Count difference: 0.1

Face detection metrics:
IoU mean: 0.633 ± 0.109
Count difference: -0.5


In [ ]:
map_calculator = MAPCalculator("annotations.xml", frame_finder, iou_threshold=0.5)

metrics = map_calculator.calculate_map()

print("Metrics:")
for class_name, value in metrics.items():
    print(f"{class_name}: {value:.4f}")

Metrics:
AP_face_bbox: 0.7712
AP_person_bbox: 0.9936
mAP: 0.8824


In [ ]:
def generate_ground_truth(frame_patterns, total_frames: int):
    pattern_frames = sorted(frame_patterns.keys())
    patterns = [frame_patterns[frame] for frame in pattern_frames]
    
    ground_truth = {}
    pattern_index = 0
    
    for frame in range(total_frames):
        if frame in frame_patterns:
            current_pattern = frame_patterns[frame]
        else:
            current_pattern = patterns[pattern_index % len(patterns)]
            pattern_index += 1
        
        ground_truth[frame] = current_pattern
    
    return ground_truth

In [ ]:
frame_patterns = {
    # 0: ["dkorolev", "mvmoiseev", "amzayakina"],
    # 0: ["ogmenkov"],
    0: ["aaivoninskaya", "avzotova_4", "anaalesuslova"],
    # 0: ["saslastnikov1"],

    
    
    
    # 15: ["ikmityushkin", "dkorolev", "mvmoiseev", "amzayakina"],
    # 30: [""dkorolev", "mvmoiseev""]
}

# total_frames = 2880
total_frames = 3600


ground_truth = generate_ground_truth(frame_patterns, total_frames)

In [ ]:
evaluator = VideoIdentificationEvaluator(
    known_identities=os.listdir("datasets/backup"),
    frame_finder=frame_finder
)

results = evaluator.process_video(ground_truth)

evaluator.print_summary(["aaivoninskaya", "avzotova_4", "anaalesuslova"])

In [16]:
import tensorflow as tf
print("Доступные GPU:", tf.config.list_physical_devices('GPU'))
print("Версия CUDA:", tf.sysconfig.get_build_info()["cuda_version"])
print("Версия cuDNN:", tf.sysconfig.get_build_info()["cudnn_version"])

Доступные GPU: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Версия CUDA: 12.4
Версия cuDNN: 9


In [1]:
import os 

os.chdir(os.path.abspath(os.path.pardir))

In [2]:
from app.models.facenet import FaceNet128dClient
import cv2
import numpy as np

def preprocess(img: np.ndarray) -> np.ndarray:
        """Предобработка изображения"""
        # Пример предобработки:
        img = cv2.resize(img, (160, 160))
        img = img.astype('float32')
        img = (img - 127.5) / 128.0  # Нормализация для FaceNet
        return np.expand_dims(img, axis=0)

model = FaceNet128dClient()


model.get_embedding(preprocess(cv2.imread("data/known_faces/ikarpova/0.png")))

2025-04-28 21:02:44.285249: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-28 21:02:44.293561: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-28 21:02:44.303680: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-28 21:02:44.306843: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-28 21:02:44.314291: I tensorflow/core/platform/cpu_feature_guar

1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step


I0000 00:00:1745863371.758816   49126 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


array([-0.333883  , -0.59391963, -0.14931333, -0.72523737,  0.17890814,
        1.0389516 , -0.1363386 ,  0.15970005,  0.76984996, -0.14774798,
        0.62146366, -0.25172126, -0.09744526, -0.65411973, -0.12484452,
        0.5087923 , -0.3434547 , -0.44630522,  0.22556825, -0.05677466,
        0.3300104 ,  0.31695297, -0.04164279,  0.5362562 ,  0.31841582,
        0.3618004 ,  0.15803212,  0.4091864 ,  0.22913374, -0.12744938,
        0.02976157,  1.3042554 , -0.5111748 ,  0.26369628, -0.0381773 ,
       -0.49421456, -0.5271343 ,  0.18327546, -0.06421107, -0.14145444,
        0.05606898, -0.22270794, -0.5373682 , -0.53704494, -0.9043109 ,
       -0.68467677,  0.50194037, -0.40308887, -0.25252044,  0.96764386,
       -0.59985167,  0.7599289 ,  0.4916856 ,  0.2005109 ,  0.3299222 ,
       -0.09368559,  0.18721025, -0.25332806, -1.2265778 ,  0.09179495,
       -0.15390548, -0.5035552 ,  0.1349945 ,  0.815584  ,  0.50403666,
       -0.08297031,  0.76674575,  1.0334882 ,  0.34060326, -0.70

In [1]:
import tensorflow as tf
# tf.debugging.set_log_device_placement(True)

with tf.device('/GPU:0'):
    a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
    b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
    c = tf.matmul(a, b)

print(c)


2025-04-28 20:22:05.206042: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-28 20:22:05.214101: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-28 20:22:05.224147: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-28 20:22:05.227252: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-28 20:22:05.234403: I tensorflow/core/platform/cpu_feature_guar

tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32)


I0000 00:00:1745860926.228959   43989 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1745860926.254630   43989 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1745860926.254769   43989 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1745860926.255457   43989 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

In [1]:
pip list | grep tensorflow

tensorflow              2.17.0
tensorflow_estimator    2.15.0
Note: you may need to restart the kernel to use updated packages.


In [2]:
import tensorflow as tf
tf.config.list_physical_devices('GPU') 


2025-04-28 00:51:11.036494: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-28 00:51:11.044435: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-28 00:51:11.054116: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-28 00:51:11.057170: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-28 00:51:11.064299: I tensorflow/core/platform/cpu_feature_guar

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
!nvcc --version  # Должно показывать 11.8

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Mar_28_02:18:24_PDT_2024
Cuda compilation tools, release 12.4, V12.4.131
Build cuda_12.4.r12.4/compiler.34097967_0
